In [1]:
import pandas as pd

In [2]:
!ls

'CSID_prelim (1).ipynb'   gnomAD_420_2023.csv
 P14410.fasta		  gnomAD_v2.1.1_ENSG00000090402_2023_01_04_01_43_24.csv
 P14410.txt		  pyproject.toml


In [3]:
# df = pd.read_csv("./gnomAD_v2.1.1_ENSG00000090402_2023_01_04_01_43_24.csv")
df = pd.read_csv("./gnomAD_420_2023.csv")


In [4]:
df.head()

,Chromosome,Position,rsIDs,Reference,Alternate,Source,Filters - exomes,Filters - genomes,Transcript,HGVS Consequence,...,Homozygote Count European (non-Finnish),Hemizygote Count European (non-Finnish),Allele Count Other,Allele Number Other,Homozygote Count Other,Hemizygote Count Other,Allele Count South Asian,Allele Number South Asian,Homozygote Count South Asian,Hemizygote Count South Asian
0,3,164748654,rs777884395,AC,A,gnomAD Exomes,PASS,NaN,ENST00000264382.3,p.Val913PhefsTer3,...,0,0,0,4022,0,0,0,22290,0,0
1,3,164748655,rs200328496,C,A,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000264382.3,p.Val913Phe,...,0,0,0,4540,0,0,0,21910,0,0
2,3,164748655,rs200328496,C,T,gnomAD Exomes,PASS,NaN,ENST00000264382.3,p.Val913Ile,...,0,0,0,3864,0,0,0,21910,0,0
3,3,164750371,rs758186369,A,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000264382.3,p.Val892Ala,...,0,0,0,7164,0,0,0,30572,0,0
4,3,164750378,rs375206629,C,T,gnomAD Exomes,PASS,NaN,ENST00000264382.3,p.Val890Ile,...,0,0,0,6074,0,0,0,30576,0,0


Variant ID is lost importing to pandas/loading in as CSV, instead of using Hail. But... I'm doing this late and I'm already too far in! So we'll just recreate variant ID, which is Chromosome-Position-Ref-Alt

In [5]:
# create a new column 'D' as the concatenated strings of columns 'A', 'B', and 'C'
df['Variant_ID'] = df['Chromosome'].astype(str) \
 + '-' + df['Position'].astype(str) \
 + '-' + df['Reference'].astype(str) \
 + '-' + df['Alternate'].astype(str)

# move the new column 'D' to the far left by reindexing the columns
df = df.reindex(columns=['Variant_ID'] + df.columns.tolist())

# display the updated dataframe
df.head()

,Variant_ID,Chromosome,Position,rsIDs,Reference,Alternate,Source,Filters - exomes,Filters - genomes,Transcript,...,Hemizygote Count European (non-Finnish),Allele Count Other,Allele Number Other,Homozygote Count Other,Hemizygote Count Other,Allele Count South Asian,Allele Number South Asian,Homozygote Count South Asian,Hemizygote Count South Asian,Variant_ID
0,3-164748654-AC-A,3,164748654,rs777884395,AC,A,gnomAD Exomes,PASS,NaN,ENST00000264382.3,...,0,0,4022,0,0,0,22290,0,0,3-164748654-AC-A
1,3-164748655-C-A,3,164748655,rs200328496,C,A,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000264382.3,...,0,0,4540,0,0,0,21910,0,0,3-164748655-C-A
2,3-164748655-C-T,3,164748655,rs200328496,C,T,gnomAD Exomes,PASS,NaN,ENST00000264382.3,...,0,0,3864,0,0,0,21910,0,0,3-164748655-C-T
3,3-164750371-A-G,3,164750371,rs758186369,A,G,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000264382.3,...,0,0,7164,0,0,0,30572,0,0,3-164750371-A-G
4,3-164750378-C-T,3,164750378,rs375206629,C,T,gnomAD Exomes,PASS,NaN,ENST00000264382.3,...,0,0,6074,0,0,0,30576,0,0,3-164750378-C-T




From gnomAD about:
Stats
v2 release is composed of 125,748 exomes and 15,708 genomes (GRCh37)
gnomAD structural variant (SV) v2.1 represents 10,847 genomes (GRCh37)
v3.1 spans 76,156 genomes (GRCh38)

... Also we get 282832 high quality allele reads in this dataset, max (this is per variant). We have a min of approx. 10k allele reads for one variant with approx 100 reads, so it's only picked up 100 times from a limited dataset, in 10k genomes that pass quality check. I think.



In [6]:
df['ClinVar Clinical Significance'].unique()

array([nan, 'Uncertain significance', 'Likely benign',
       'Conflicting interpretations of pathogenicity',
       'Benign/Likely benign', 'Benign', 'Likely pathogenic',
       'Pathogenic', 'Pathogenic/Likely pathogenic'], dtype=object)

In [7]:
df['ClinVar Variation ID'].count()


638

457 documented variants in ClinVar -- many of unknown/conflicting significance.

In [8]:
df_path = df[df['ClinVar Clinical Significance'].isin(['Pathogenic',
                                                  'Pathogenic/Likely pathogenic',
                                                  'Likely pathogenic'
                                                  ])]

In [9]:
df_path['ClinVar Variation ID'].count()

29

18 variants, of the 457 recorded in ClinVar, are currently clinically annotated. Note this is for variants caught in ClinVar -- many of which are not annotated. There are many variants not in ClinVar... just starting off small.

If we now however look at the predicted effects for variants from df, we'll see more. VEP looks at genetic conservation, effects on regulatory regions if involved, and ultimately the difference in transcript/protein. It does not dock.

See also:
ClinVar:
https://www.ncbi.nlm.nih.gov/clinvar/?gr=0&term=SI[gene]

dbVar (includes structural vs. clinical):
https://www.ncbi.nlm.nih.gov/dbvar?LinkName=gene_dbvar&from_uid=6476


THIS NEXT ONE GETS ALL THE VARIANTS!

In [10]:
# Let's get full numbers before the following:
print("the total unique variants in df")
df['Variant_ID'].count()

the total unique variants in df


Variant_ID    3470
Variant_ID    3470
dtype: int64

In [11]:
print("the total variants recorded... with clinvar pathogenic annotation")
df_path['Variant_ID'].count()

the total variants recorded... with clinvar pathogenic annotation


Variant_ID    29
Variant_ID    29
dtype: int64

In [12]:
df['VEP Annotation'].unique()

array(['frameshift_variant', 'missense_variant', 'synonymous_variant',
       'splice_region_variant', 'inframe_deletion', 'stop_gained',
       'start_lost', 'protein_altering_variant', 'intron_variant',
       'splice_acceptor_variant', 'splice_donor_variant',
       '3_prime_UTR_variant'], dtype=object)

In [13]:
df_vep = df[df['VEP Annotation'].isin(['frameshift_variant',
                                       'stop_gained',
                                       'splice_donor_variant',
                                       'splice_acceptor_variant',
                                       ])]

In [14]:
df_vep['Variant_ID'].count()

Variant_ID    168
Variant_ID    168
dtype: int64

THAT'S THE BIG NUMBER! 168 variants that are predicted based off VEP. 150 new ones that could lead to some effect, at least based off the limited data in ensembl.

In [15]:
df_vep['Homozygote Count'].sum()

0

In [16]:
df_path['Homozygote Count'].sum()


0

This is really interesting. CSID occurs in patients that are homozygote and neither allele produces functional sucrase. There's also literature that suggests heterozygote individuals also experience symptoms. It may be that no one in gnomAD has CSID, validating prior work on approx. 1/400,000 people having CSID. Just works out they weren't featured in the approx 100,000 genomes.

... But gnomAD also specifically states they try not to include patients with rare diseases, because their specific mutation can potentially be used to identify them/family.

In [17]:
# let's instead look at how many have these variants
df_vep['Allele Count'].sum()

401

So, 200 heterozygotes with one allele predicted bad. 
200/150,000 = 1/750 (0.13%) heterozygote with potential symptoms

Current estimate for homozygote approx. 1/400,000: 
de Leusse, C., et al. "Estimating the prevalence of congenital disaccharidase deficiencies using allele frequencies from gnomAD." Archives de Pédiatrie 29.8 (2022): 599-603.

But yeah, homozygote count is hidden/0 in this genetic database.

---

Most missense are benign for this disease; the ones that are pathogenic or likely pathogenic are above. These could still be modeled. The number is like 102k missense for one "variant" which may as well be the reference. 

In [18]:
df_vep_mis = df[df['VEP Annotation'].isin([
                                       'missense_variant',
                                       ])]

In [19]:
df_vep_mis['Variant_ID'].count()


Variant_ID    1141
Variant_ID    1141
dtype: int64

In [20]:
df_vep_mis['Homozygote Count'].describe()

count      1141.000000
mean        145.155127
std        3426.004690
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max      102305.000000
Name: Homozygote Count, dtype: float64

In [21]:
# this is weird, another day
df_vep_mis['Homozygote Count'].sum()

165622

---
Manufacture the mutant sequence!

This will involve... deltaSeq = fn(pos, variant, refSeq)
Probably use biopython, hope it reliably alters the sequence

In [22]:
df_path.head()


,Variant_ID,Chromosome,Position,rsIDs,Reference,Alternate,Source,Filters - exomes,Filters - genomes,Transcript,...,Hemizygote Count European (non-Finnish),Allele Count Other,Allele Number Other,Homozygote Count Other,Hemizygote Count Other,Allele Count South Asian,Allele Number South Asian,Homozygote Count South Asian,Hemizygote Count South Asian,Variant_ID
247,3-164737547-C-T,3,164737547,rs774109373,C,T,gnomAD Exomes,PASS,NaN,ENST00000264382.3,...,0,1,6108,0,0,0,30608,0,0,3-164737547-C-T
354,3-164712069-G-A,3,164712069,rs376062850,G,A,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000264382.3,...,0,1,7182,0,0,0,30594,0,0,3-164712069-G-A
413,3-164741510-AAG-A,3,164741510,rs751263307,AAG,A,gnomAD Exomes,PASS,NaN,ENST00000264382.3,...,0,0,6090,0,0,0,30604,0,0,3-164741510-AAG-A
697,3-164750402-CAAGGATTT-C,3,164750402,rs1461489544,CAAGGATTT,C,gnomAD Exomes,PASS,NaN,ENST00000264382.3,...,0,0,6082,0,0,0,30574,0,0,3-164750402-CAAGGATTT-C
779,3-164781249-T-TA,3,164781249,rs769103567,T,TA,"gnomAD Exomes,gnomAD Genomes",PASS,PASS,ENST00000264382.3,...,0,0,7110,0,0,0,30114,0,0,3-164781249-T-TA


---
This data table is not reasonable for this task, however. I think so far it's helped and provided more info than the default table in gnomAD for getting numbers, but I really now just want the protein seq, can back track to this later. So, we need to reconfigure our table.


In [23]:
pathVarPos = df_path['HGVS Consequence']

In [24]:
# I'm showing this to demo the protein and codon level changes
pathVarPos

247             p.Trp1089Ter
354             p.Thr1606Ile
413       p.Ser982LeufsTer16
697       p.Lys879ArgfsTer10
779        p.Leu296PhefsTer4
995         p.Gly92LeufsTer8
1119             p.Glu613Ter
1128             p.Glu285Ter
1135             p.Glu179Ter
1187       p.Gln677AlafsTer7
1203             p.Gln278Ter
1204      p.Gln278HisfsTer18
1244      p.Cys646AlafsTer44
1253            p.Cys1229Tyr
1284             p.Asp536Val
1448      p.Asn1403IlefsTer8
1462      p.Asn1027LysfsTer2
1510             p.Arg263Ter
1537            p.Arg1609Ter
1550            p.Arg1504Ter
1576            p.Arg1124Ter
1589            p.Arg1077Ter
1824              c.635+2T>C
2091    c.4840_4841+2delGAGT
2206             c.4575+2T>C
2524              c.373+1G>A
2787             c.2737-1G>C
3229             c.1513-1G>A
3386             c.1146+1G>A
Name: HGVS Consequence, dtype: object

Alright, so, I need to filter this for p*; I'll double check later if these are exclusive or not, i.e. if c and p are recorded, nto the same mutation. For now just want some proof of concept. And... I think I wanna grab the frameshift ones first. SOOO....

In [25]:
df_frameshift = df[df['VEP Annotation'].isin([
                                       'frameshift_variant',
                                       ])]
df_frameshift.head()

,Variant_ID,Chromosome,Position,rsIDs,Reference,Alternate,Source,Filters - exomes,Filters - genomes,Transcript,...,Hemizygote Count European (non-Finnish),Allele Count Other,Allele Number Other,Homozygote Count Other,Hemizygote Count Other,Allele Count South Asian,Allele Number South Asian,Homozygote Count South Asian,Hemizygote Count South Asian,Variant_ID
0,3-164748654-AC-A,3,164748654,rs777884395,AC,A,gnomAD Exomes,PASS,NaN,ENST00000264382.3,...,0,0,4022,0,0,0,22290,0,0,3-164748654-AC-A
12,3-164755783-C-CCGTT,3,164755783,rs544348891,C,CCGTT,gnomAD Exomes,PASS,NaN,ENST00000264382.3,...,0,0,6108,0,0,0,30604,0,0,3-164755783-C-CCGTT
27,3-164764795-A-ACAGC,3,164764795,rs1273409266,A,ACAGC,gnomAD Exomes,PASS,NaN,ENST00000264382.3,...,0,0,5852,0,0,0,29134,0,0,3-164764795-A-ACAGC
159,3-164783146-T-TA,3,164783146,NaN,T,TA,gnomAD Exomes,PASS,NaN,ENST00000264382.3,...,0,0,6110,0,0,0,30584,0,0,3-164783146-T-TA
162,3-164785190-ATCAT-A,3,164785190,rs927354199,ATCAT,A,gnomAD Exomes,PASS,NaN,ENST00000264382.3,...,0,0,6126,0,0,1,30606,0,0,3-164785190-ATCAT-A


In [26]:
frameshiftPos = df_frameshift['HGVS Consequence']
frameshiftPos

0         p.Val913PhefsTer3
12        p.Val778ThrfsTer3
27        p.Val574GlyfsTer8
159      p.Tyr237LeufsTer11
162       p.Tyr190LeufsTer2
205            p.Tyr1063Ter
226             p.Trp453Ter
281      p.Thr725ArgfsTer22
350     p.Thr1661ArgfsTer14
405     p.Thr1130IlefsTer54
413      p.Ser982LeufsTer16
437       p.Ser516LeufsTer7
469     p.Ser1723LeufsTer10
497     p.Ser1012LeufsTer11
501             p.Pro898Ter
517      p.Pro579LeufsTer24
532      p.Pro409LeufsTer24
581     p.Pro1140LeufsTer44
603      p.Phe927LeufsTer69
608       p.Phe831LeufsTer3
618      p.Phe411LeufsTer22
644     p.Phe1521GlufsTer62
686     p.Met1196ValfsTer15
697      p.Lys879ArgfsTer10
707         p.Lys4AsnfsTer9
713      p.Lys193IlefsTer22
754      p.Leu735PhefsTer12
759       p.Leu681TrpfsTer9
779       p.Leu296PhefsTer4
780       p.Leu296PhefsTer2
794     p.Leu1772PhefsTer16
795     p.Leu1772GlyfsTer20
843       p.Leu102TyrfsTer8
963       p.His451ArgfsTer5
984      p.His1210GlnfsTer2
995        p.Gly92Le

Ok, that's kinda misleading, but it's what we're going for. Now need to assure only p* in these values

And because I was thinking ahead to getting the coordinates: We can definitely use positions 1500 and 1394, highly conserved D, aspartic acid (asparDate) to extract coordinates wherever alphafold spits out the residues. Then autobox from there. BAM! Or specify the gridbox etc. 

"Asp1394 in SUC within the conserved motif of DGLWIDMNE (underlined) constitute the catalytic nucleophile residues in the active site (27, 28). By resolving the crystal structure of the N-terminal domain of SI..."

he nucleophilic catalytic residue of human SI is located in the DGLWIDMNE sequence, which is conserved among all members of the GH31 family of enzymes (12, 28, 31). By resolving the crystal structure of the N-terminal domain of human SI, Asp604

Therefore, based on amino acid sequence alignment, we predicted Asp1500 in the consensus sequence of HWLGDN in SUC to function as the proton donor residue for hydrolysis (Fig. 2). "
https://www.jbc.org/article/S0021-9258(20)36518-2/fulltext
Gericke, Birthe, et al. "Structure-function analysis of human sucrase-isomaltase identifies key residues required for catalytic activity." Journal of Biological Chemistry 292.26 (2017): 11070-11078.

Their thing is based off: https://www.rcsb.org/structure/3TOP


---
ANYWAY, I still need wild type human sucrase sequence.
Ladies and gentlemen, we found her:
https://www.uniprot.org/uniprotkb/P14410/entry
https://rest.uniprot.org/uniprotkb/P14410.fasta

https://grch37.ensembl.org/Homo_sapiens/Transcript/Summary?g=ENSG00000090402;r=3:164696686-164796283;t=ENST00000264382

.txt also available; it states 1827 AA. 

Import seq from ref fasta file

In [27]:
# Copied those both in. Now we need Biopython... I suppose, could also do with strings but that'd be annoying
from Bio import SeqIO

# open the FASTA file
with open("P14410.fasta") as handle:
    # parse the file using SeqIO
    for record in SeqIO.parse(handle, "fasta"):
        # access the sequence and description of each record
        sequence = record.seq
        description = record.description
        print(f"{description}: {sequence}")


ModuleNotFoundError: No module named 'Bio'

In [ ]:
from Bio.Seq import Seq

# Define the reference protein sequence
ref_prot_seq = Seq("MVLSPADKTNVKAAWGKVGAHAGEYGAEALERMFLSFPTTKTYFPHFDLSHGSAQVKGHGKKVADALTNAVAHVDDMPNALSALSDLHAHKLRVDPVNFKLLSHCLLVTLAAHLPAEFTPAVHASLDKFLASVSTVLTSKYR")

# Define the mutation position and mutation type
mutation_pos = 35
mutation_type = "frameshift"

# Translate the protein sequence to DNA
ref_dna_seq = ref_prot_seq.back_transcribe()

# Insert the frameshift mutation
if mutation_type == "frameshift":
    upstream_seq = ref_dna_seq[:3 * (mutation_pos - 1)]
    frameshift_seq = Seq("AAA")
    downstream_seq = ref_dna_seq[3 * (mutation_pos - 1):]
    mutated_dna_seq = upstream_seq + frameshift_seq + downstream_seq

# Translate the mutated DNA sequence to protein
mutated_prot_seq = mutated_dna_seq.translate()
